<a href="https://colab.research.google.com/github/Agustin-Galarza/TP2-NLP-Grupo-5---code2bert/blob/main/code2bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# code2vec

Download our version of code2vec

In [ ]:
!git clone https://github.com/Agustin-Galarza/code2vec

fatal: destination path 'code2vec' already exists and is not an empty directory.


(Just in case): remove the project

In [ ]:
#!rm -rf code2vec

Import the .java files' dataset from code2vec

In [ ]:
!wget https://s3.amazonaws.com/code2seq/datasets/java-small.tar.gz
!tar -xzf java-small.tar.gz -C ./code2vec/
#!rm -rf code2vec/input_data
!mv code2vec/java-small code2vec/input
!rm java-small.tar.gz

--2023-11-11 19:19:24--  https://s3.amazonaws.com/code2seq/datasets/java-small.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.124.176, 52.217.164.216, 52.216.44.152, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.124.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83611808 (80M) [application/x-tar]
Saving to: ‘java-small.tar.gz’

java-small.tar.gz   100%[===================>]  79.74M  41.1MB/s    in 1.9s    

2023-11-11 19:19:26 (41.1 MB/s) - ‘java-small.tar.gz’ saved [83611808/83611808]



Reduce the amount of files so we don't have memory issues.

Deleting 10% of the files works, but it takes too long, so we should test further.
With 80% we end up getting 24k functions (that in reallity are even more because it doesn't sepparate the functions per line very well yet), so we should be using values around 90%

In [ ]:
import os
from random import random

base_input_dir = "code2vec/input"
input_dirs = [f"{base_input_dir}/test", f"{base_input_dir}/training", f"{base_input_dir}/validation"]
delete_prob = 0.90

for input_dir in input_dirs:
    for dir in os.listdir(input_dir):
        dir_path = os.path.join(input_dir, dir)
        for file in os.listdir(dir_path):
            lottery = random()
            if lottery < delete_prob:
                file_path = os.path.join(dir_path, file)
                os.remove(file_path)

Delete the previous (or default) output data

In [ ]:
!rm code2vec/output_data/*

Run the preprocessing job for the dataset.

Remember to change the variables in preprocess.sh so
that the program scans the downloaded dataset files.
Right now, the downloaded Java dataset is stored in `code2vec/input`, but the default values in `process.sh` and `process.short.sh` *might be different*.
Also, watch out for the different sub directories, which are called **training**, **validation** and **test** in the downloaded dataset, but the current defaults are _train_, _val_ and _test_ (we need to change this)

In [ ]:
!cd code2vec; ./preprocess.short.sh

Extracting paths from validation set...
Finished extracting paths from validation set
Extracting paths from test set...
Finished extracting paths from test set
Extracting paths from training set...
Finished extracting paths from training set


Move the output raw files to the root so we can build the dataset

In [ ]:
!mv ./code2vec/output_data ./

# Download the raw methods directly

In [ ]:
!git clone https://github.com/Agustin-Galarza/code2vec ourcode2vec
!mv ./ourcode2vec/output_data ./
!rm -rf ourcode2vec

Cloning into 'ourcode2vec'...
remote: Enumerating objects: 821, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 821 (delta 40), reused 96 (delta 27), pack-reused 703
Receiving objects: 100% (821/821), 9.69 MiB | 19.09 MiB/s, done.
Resolving deltas: 100% (455/455), done.
mv: cannot move './ourcode2vec/output_data' to './output_data': Directory not empty


# Bert

In [ ]:
!pip install transformers==4.34.1 datasets==2.14.5 accelerate

## Create Dataset

### From files build Dict

In [ ]:
from datasets import Dataset, DatasetDict
from functools import partial

In [ ]:
raw_files_dir = "output_data"

def dataset_generator(filename):
    with open(filename, "r") as file:
        print(file)
        for line in file:
            yield {"text": line}

train_gen = partial(dataset_generator, filename=f"./{raw_files_dir}/train")
test_gen = partial(dataset_generator, filename=f"./{raw_files_dir}/test")
val_gen = partial(dataset_generator, filename=f"./{raw_files_dir}/val")

In [ ]:
train_dataset = Dataset.from_generator(train_gen)
test_dataset = Dataset.from_generator(test_gen)
val_dataset = Dataset.from_generator(val_gen)

ds = DatasetDict({"train": train_dataset, "test": test_dataset, "val": val_dataset})

Generating train split: 0 examples [00:00, ? examples/s]

<_io.TextIOWrapper name='./output_data/train' mode='r' encoding='UTF-8'>


Generating train split: 0 examples [00:00, ? examples/s]

<_io.TextIOWrapper name='./output_data/test' mode='r' encoding='UTF-8'>


Generating train split: 0 examples [00:00, ? examples/s]

<_io.TextIOWrapper name='./output_data/val' mode='r' encoding='UTF-8'>


In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18828
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1419
    })
    val: Dataset({
        features: ['text'],
        num_rows: 1889
    })
})


In [ ]:
max_out = 30
for i, data in enumerate(ds["train"]):
    if i == max_out: break
    print(data)

{'text': 'public String getSchemaName() {return schemaName;}public Date getDate() {return date;}protected boolean excludeEmpty() {return true;}public EntityPersister getSource() {return (EntityPersister) super.getSource();}public long getId() {return id;}\n'}
{'text': 'public void setId(long id) {this.id = id;}java.lang.String getId() {return id;}\n'}
{'text': 'void setId(java.lang.String newValue) {id = newValue;}public Object read(ResultSet resultSet, ResultSetProcessingContext context) {//To change body of implemented methods use File | Settings | File Templates.return null;}public String getPayload() {return payload;}\n'}
{'text': 'public void setPayload(String payload) {this.payload = payload;}public Long getId() {return 1L;}\n'}
{'text': 'public void setId(Long id) {}\n'}
{'text': 'public Car getCar() {return null;}\n'}
{'text': 'public void setCar(Car car) {}public final void setGroupId(long groupId) {this.groupId = groupId;}\n'}
{'text': 'protected final long getGroupId() {retu

## Load the tokenizer

In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name, fast=True)

In [ ]:
print(tokenizer.mask_token)
print("Fast: ", tokenizer.is_fast)
print(tokenizer.model_max_length)

[MASK]
Fast:  True
512


## Tokenize the Dataset

In [ ]:
def tokenize_document(document_batch):
    res = tokenizer(document_batch["text"])
    res["word_ids"] = [res.word_ids(i) for i in range(len(res["input_ids"]))]
    return res

In [ ]:
tokenized_dataset = ds.map(
    tokenize_document, batched=True, remove_columns=["text"]
)

Map:   0%|          | 0/18828 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1419 [00:00<?, ? examples/s]

Map:   0%|          | 0/1889 [00:00<?, ? examples/s]

Max length for document in tokenized dataset

In [ ]:
from statistics import mean, median
lens = []
for doc in tokenized_dataset["train"]["input_ids"]:
    if type(len) is int:
        print(len)
        break
    lens.append(len(doc))

In [ ]:
print(f"Max length: {max(lens)}")
print(f"Mean length: {mean(lens)}")
print(f"Median length: {median(lens)}")
print(f"Docs with more than 256 tokens: {len([1 for i in lens if i > 256])} out of {len(lens)}")

Max length: 8848
Mean length: 139.13867643934566
Median length: 69.0
Docs with more than 256 tokens: 2659 out of 18828


Concatenate inputs for batch processing

In [ ]:
chunk_size = 256

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // chunk_size) * chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_dataset.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/18828 [00:00<?, ? examples/s]

Map:   0%|          | 0/1419 [00:00<?, ? examples/s]

Map:   0%|          | 0/1889 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10225
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 851
    })
    val: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 809
    })
})

## Fine Tune Bert

### Define dataset to use
Downsample dataset to get faster results

In [ ]:
train_size = 10_000
test_train_proportion = 0.03
test_size = int(test_train_proportion * train_size)

print(test_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

300


ValueError: ignored

### Build data collator
Be careful with the `batch_size`, since a big value can overflow the ram in nanoseconds. With a batch_size = 10, the training uses 6.9GB of VRAM in total, so we could maybe handle a value around 15-20.

In [ ]:
import collections
import numpy as np

from transformers import (
    default_data_collator, DataCollatorForLanguageModeling, get_scheduler, AutoModelForMaskedLM
    )
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate import Accelerator

mask_probability = 0.15
whole_word_masking_probability = 0.15
masking_type = "whole_word"
num_train_epochs = 14
batch_size = 25


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=mask_probability
)

Or we could use a whole word data collator

In [ ]:
def removeNone(examples):
    return {
        k: [v if v is not None else 0 for v in examples[k]] for k in examples.keys()
    }

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, whole_word_masking_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels
    # Sanitize data
    for f, feature in enumerate(features):
        for key in feature.keys():
            copied_values = []
            for i, val in enumerate(feature[key]):
                copied_values.append(val if val is not None else 0)
                feature[key] = copied_values
    # Check
    for feature in features:
        for key in feature.keys():
            for i, val in enumerate(feature[key]):
                if val is None:
                    print("Problem")
    return default_data_collator(features)

In [ ]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    match masking_type:
        case "default":
            masked_inputs = data_collator(features)
        case "whole_word":
            masked_inputs = whole_word_masking_data_collator(features)
        case _:
            raise Exception("Invalid type")
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [ ]:
if masking_type != "whole_word":
    downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])

eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

# Remove the word_ids column after the masking is done because the Dataloader
# does not accept it
if masking_type == "whole_word":
    downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])

eval_dataset = eval_dataset.remove_columns(["masked_token_type_ids"])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_dataloader = DataLoader(
    downsampled_dataset["train"].with_format(type="torch"),
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset.with_format(type="torch"),
    batch_size=batch_size,
    collate_fn=default_data_collator
)

In [ ]:
print(train_dataloader)
print(downsampled_dataset["train"].with_format(type="torch"),)
print(eval_dataset.with_format(type="torch"))

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})


## Download model

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_name)
optimizer = AdamW(model.parameters(), lr=5e-5)

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

## Login to Huggingface
This is for uploading the model to Huggingface

currently it is uploaded in `agalarza/bert-base-uncased-finetuned-java-methods`

In [ ]:
from huggingface_hub import get_full_repo_name, Repository, notebook_login

In [ ]:
notebook_login()

In [ ]:
own_model_name = "bert-base-uncased-finetuned-java-methods"
# own_model_name = "bert-nlp"
repo_name = get_full_repo_name(own_model_name)
print("Repo name:", repo_name)

output_dir = own_model_name
repo = Repository(output_dir, clone_from=repo_name)

Cloning https://huggingface.co/agalarza/bert-nlp into local empty directory.


Repo name: agalarza/bert-nlp


OSError: ignored

## Train

In [ ]:
import torch
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [ ]:
from tqdm.auto import tqdm
import torch
import math


progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f"- Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if repo is not None and accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/5600 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Empty the cache to free the memory

In [ ]:
torch.cuda.empty_cache()

# Load trained model
Use our pre-trained model

In [ ]:
print(repo_name)

Benve/bert-nlp


Install modules if not already installed

In [ ]:
!pip install transformers==4.34.1

In [ ]:
from transformers import pipeline

output_dir = "Benve/bert-nlp"
mask_filler = pipeline("fill-mask", model=output_dir)

In [ ]:
example_functions = [
    f"""
    public void {tokenizer.mask_token}(String[] args) {{
        System.out.println(\"Hello World!\");
    }}
    """,
    f"""
    public int {tokenizer.mask_token}(int x, int y) {{
        return x + y;
    }}
    """,
    f"""
    @Override public static int {tokenizer.mask_token}(List<Integer> list) {{
        int current = -1;
        for(Integer e : list) {{
            if(e > current) {{
                current = e;
            }}
        }}
        return current;
    }}
    """,
]

predictions = mask_filler(masked_validate_functions[0])



[{'score': 0.2620081305503845, 'token': 1243, 'token_str': 'get', 'sequence': '[CLS] protected static long get ( SWIGTYPE _ p _ p _ btSoftBody obj ) { return ( obj = = null )? 0 : obj. swigCPtr ; } protected static long [MASK] ( SWIGTYPE _ p _ PHY _ ScalarType obj ) { return ( obj = = null )? 0 : obj. swigCPtr ; } public MeasuredDimension calcMeasures ( int widthMeasureSpec, int heightMeasureSpec ) { return new MeasuredDimension ( width, height ) ; } public boolean isDone ( ) { return future. isDone ( ) ; } [SEP]'}, {'score': 0.04808776453137398, 'token': 2373, 'token_str': 'read', 'sequence': '[CLS] protected static long read ( SWIGTYPE _ p _ p _ btSoftBody obj ) { return ( obj = = null )? 0 : obj. swigCPtr ; } protected static long [MASK] ( SWIGTYPE _ p _ PHY _ ScalarType obj ) { return ( obj = = null )? 0 : obj. swigCPtr ; } public MeasuredDimension calcMeasures ( int widthMeasureSpec, int heightMeasureSpec ) { return new MeasuredDimension ( width, height ) ; } public boolean isDone

In [ ]:
def flatten(l):
    match l:
        case (list() | tuple() | set()) as items:
            for item in items:
                yield from flatten(item)
        case _:
            yield l

# Prepare code2vec

In [ ]:
!wget https://s3.amazonaws.com/code2vec/model/java14m_model.tar.gz
!tar -xvzf java14m_model.tar.gz
!rm java14m_model.tar.gz

--2023-11-12 04:06:18--  https://s3.amazonaws.com/code2vec/model/java14m_model.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.117.248, 54.231.168.216, 52.217.204.40, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.117.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1440921240 (1.3G) [application/x-tar]
Saving to: ‘java14m_model.tar.gz’

java14m_model.tar.g 100%[===================>]   1.34G  37.8MB/s    in 37s     

2023-11-12 04:06:56 (36.7 MB/s) - ‘java14m_model.tar.gz’ saved [1440921240/1440921240]

models/java14_model/saved_model_iter8.release.data-00000-of-00001
models/java14_model/saved_model_iter8.release.index
models/java14_model/saved_model_iter8.release.meta
models/java14_model/dictionaries.bin


In [ ]:
!rm -rf code2vec
!git clone https://github.com/tech-srl/code2vec.git

Cloning into 'code2vec'...
remote: Enumerating objects: 718, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 718 (delta 2), reused 8 (delta 0), pack-reused 704
Receiving objects: 100% (718/718), 5.14 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (418/418), done.


In [ ]:
#!cd code2vec; python3 code2vec.py --load ../models/java14_model/saved_model_iter8.release --predict

In [ ]:
import sys
if not "/content/code2vec" in sys.path:
    sys.path.append("/content/code2vec")
if not "./code2vec" in sys.path:
    sys.path.append("./code2vec")


from extractor import Extractor
from common import common
from vocabularies import Code2VecVocabs, VocabType
from config import Config
from model_base import Code2VecModelBase
from tensorflow_model import Code2VecModel

SHOW_TOP_CONTEXTS = 10
MAX_PATH_LENGTH = 8
MAX_PATH_WIDTH = 2
JAR_PATH = 'code2vec/JavaExtractor/JPredict/target/JavaExtractor-0.0.1-SNAPSHOT.jar'

# tensorflow throws an error if we re-instantiate the model, so we have to prevent that
if (not 'config' in locals() and not 'config' in globals()) or config is None:
    config = Config(set_defaults=True)

    # Load arguments BY HAND
    config.MODEL_LOAD_PATH = "models/java14_model/saved_model_iter8.release"
    config.DL_FRAMEWORK = "tensorflow"

    c2v_model = Code2VecModel(config)

    path_extractor = Extractor(config,
                            jar_path=JAR_PATH,
                            max_path_length=MAX_PATH_LENGTH,
                            max_path_width=MAX_PATH_WIDTH)

def code2vec_prediction(code: str, print=False):
    path_to_input = "./Input.java"
    with open(path_to_input, "w") as file:
        file.write(code)
    try:
        predict_lines, hash_to_string_dict = path_extractor.extract_paths(path_to_input)
    except ValueError as e:
        print("Error: ", e)
        return
    raw_prediction_results = c2v_model.predict(predict_lines)
    method_prediction_results = common.parse_prediction_results(
        raw_prediction_results, hash_to_string_dict,
        c2v_model.vocabs.target_vocab.special_words, topk=SHOW_TOP_CONTEXTS)

    if print:
        for raw_prediction, method_prediction in zip(raw_prediction_results, method_prediction_results):
            print('Original name:\t' + method_prediction.original_name)
            for name_prob_pair in method_prediction.predictions:
                print('\t(%f) predicted: %s' % (name_prob_pair['probability'], name_prob_pair['name']))
            print('Attention:')
            for attention_obj in method_prediction.attention_paths:
                print('%f\tcontext: %s,%s,%s' % (
                attention_obj['score'], attention_obj['token1'], attention_obj['path'], attention_obj['token2']))
            if config.EXPORT_CODE_VECTORS:
                print('Code vector:')
                print(' '.join(map(str, raw_prediction.code_vector)))

    predictions = list(map(lambda x: x.predictions, method_prediction_results))[0]
    return predictions

# Generate and evaluate predictions


First define an interface for the prediction results.

Each prediction will be just a list of tokens and the assigned probability.

We also have to define methods to transform the results from both our model and code2vec into this interface.

In [ ]:
from typing import List, Tuple
from dataclasses import dataclass

def join_name(word_list: List[str]) -> str:
    for i, r in enumerate(word_list):
        if i > 0:
            word_list[i] = r.capitalize()
    return "".join(word_list)

@dataclass
class Prediction:
    value: List[str]
    probability: float

    def __repr__(self):
        return f"(value: {self.value} | probability: {self.probability:.2f})"

    def cammel_case_value(self) -> str:
        return join_name(self.value)

In [ ]:
def filter_own_pred(own_pred) -> List:
    # Some lines are not correctly parsed and they contain more than one method
    # in these cases there could be more than one mask and more than one prediction,
    # so we have to take into account only the first one
    if type(own_pred[0]) == list:
        return own_pred[0]
    return own_pred


def parse_our_prediction(pred) -> List[Prediction]:
    pred = filter_own_pred(pred)
    return list(map(lambda p: Prediction([p['token_str']], p['score']), pred))

def parse_c2v_prediction(pred) -> List[Prediction]:
    return list(map(lambda p: Prediction(p[0], p[1]), pred))

## Run the evaluation

In [ ]:
!pip install jsonlines

In [ ]:
import re
from tqdm import tqdm
import json
import jsonlines
from typing import List

filepath = "./output_data/val"

max_results = None
batch_saving = False
batch_saving_interval = 50
batch_filename = "./batch_save.jsonl"

masked_validate_functions: List[str] = []
method_names: List[str] = []
original_methods: List[str] = []
c2v_parsed_predictions: List[List[Prediction]] = []
our_parsed_predictions: List[List[Prediction]] = []

line_count = sum([1 for _ in open(filepath)])

successful_results = 0
batch_file = None
if batch_saving:
    batch_file = jsonlines.open(batch_filename, "a")

with open(filepath, "r") as file:
  for line in tqdm(file, total=line_count, desc="Processing data"):
    if batch_saving and successful_results % batch_saving_interval == 0:
        serialized_c2v = [list(map(
            lambda x: dict(value=x.value, probability=x.probability),
            pred
        )) for pred in c2v_parsed_predictions]

        serialized_our = [list(map(
            lambda x: dict(value=x.value, probability=x.probability),
            pred
        )) for pred in our_parsed_predictions]
        batch_file.write(dict(
            masked_validate_functions=masked_validate_functions,
            method_names=method_names,
            original_methods=original_methods,
            c2v_parsed_predictions=serialized_c2v,
            our_parsed_predictions=serialized_our,
        ))
    if max_results is not None and successful_results == max_results:
        break
    counter = 2
    splited_line = line.split(" ")

    if "static" in splited_line:
      counter +=1
    if "synchronized" in splited_line:
      counter +=1

    clean_function_name = re.sub(r'\(.*', '',splited_line[counter])

    masked_function = line.replace(clean_function_name, tokenizer.mask_token)

    if len(masked_function) > tokenizer.model_max_length:
        continue

    try:
        c2v_prediction = [(p['name'], p['probability']) for p in code2vec_prediction(line)]
        our_prediction = mask_filler(masked_function)
    except:
        continue

    original_methods.append(line)
    method_names.append(clean_function_name)
    masked_validate_functions.append(masked_function)
    c2v_parsed_predictions.append(parse_c2v_prediction(c2v_prediction))
    our_parsed_predictions.append(parse_our_prediction(our_prediction))

    successful_results += 1

if batch_file is not None:
    batch_file.close()

Processing data:  49%|████▉     | 976/1977 [14:48<15:11,  1.10it/s]


In [ ]:
# # Load the saved data

# masked_validate_functions: List[str] = []
# method_names: List[str] = []
# original_methods: List[str] = []
# c2v_parsed_predictions: List[List[Prediction]] = []
# our_parsed_predictions: List[List[Prediction]] = []

# line_count = sum([1 for _ in open(batch_filename)])

# with jsonlines.open(batch_filename, "r") as file:
#     # We just need to load the last entry
#     for i in range(line_count):
#         try:
#             line = file.read()
#         except:
#             continue
#         if i == line_count - 1:
#             masked_validate_functions = line.get("masked_validate_functions", [])
#             method_names = line.get("method_names", [])
#             original_methods = line.get("original_methods", [])
#             c2v_parsed_predictions = line.get("c2v_parsed_predictions", [])
#             our_parsed_predictions = line.get("our_parsed_predictions", [])

850
850
850
850
850


In [ ]:
len(original_methods)

900

## Use word2vec for comparison

In [ ]:
!pip install -U -q gensim

In [ ]:
import gensim.downloader

w2v_model = gensim.downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
from typing import Tuple, List
import re

def parse_cammel_case(word: str) -> List[str]:
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', word)
    return [m.group(0) for m in matches]

def score_prediction_result(
    prediction_result: List[Prediction],
    method_name: List[str]
    ) -> List[float]:
    return [w2v_model.n_similarity(pred.value, method_name) for pred in prediction_result]

def weighted_score_prediction_result(
    prediction_result: List[Prediction],
    method_name: List[str]
) -> float:
    scores = score_prediction_result(prediction_result, method_name)
    return sum([p.probability*s for p,s in zip(prediction_result, scores)])

In [ ]:
parse_cammel_case(method_names[0])

['get', 'c', 'ptr']

In [ ]:
score = score_prediction_result(our_parsed_predictions[0], parse_cammel_case(method_names[0]))
print("Prediction Score:", score)

print("Probabilities:", [p.probability for p in our_parsed_predictions[0]])
print("Weighted:", weighted_score_prediction_result(our_parsed_predictions[0], parse_cammel_case(method_names[0])))

Prediction Score: [1.0, 0.5726487, 0.46117607, 0.40115592, 0.61345124]
Probabilities: [0.2620081305503845, 0.04808776453137398, 0.03511582687497139, 0.027507711201906204, 0.023317234590649605]
Weighted: 0.33107897323077995


In [ ]:
score = score_prediction_result(c2v_parsed_predictions[0], parse_cammel_case(method_names[0]))
print("Prediction Score:", score)

print("Probabilities:", [p.probability for p in c2v_parsed_predictions[0]])
print("Weighted:", weighted_score_prediction_result(c2v_parsed_predictions[0], parse_cammel_case(method_names[0])))

Prediction Score: [1.0, 0.43109724, 1.0, 1.0, 1.0, 1.0, 1.0, 0.30686238, 1.0, 1.0]
Probabilities: [1.0, 1.054314235915399e-08, 8.130188966504193e-09, 4.1625649593868275e-09, 2.1514112713560962e-09, 1.9387691452266154e-09, 1.7844875577210928e-09, 6.839201427411012e-10, 3.1865962646371315e-10, 3.1842023462402835e-10]
Weighted: 1.0000000235594908


# Generate dataset for analisys

In [ ]:
from pandas import DataFrame
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class PredictionResult:
    method_str: str # El string completo del método a evaluar
    masked_method: str # El string del método con la máscara aplicada
    method_name: str # El nombre original del método
    prediction: List[Prediction] # La lista de predicciones obtenidas por nuestro Bert
    prediction_scores: List[float] # Los scores de similitud con el nombre original de nuestras predicciones
    prediction_weighted_score: float # La suma de los scores, ponderada por la probabilidad de cada predicción
    c2v_pred: List[Prediction] # La lista de predicciones obtenidas por code2vec
    c2v_prediction_scores: List[float] # La suma de los scores, ponderada por la probabilidad de cada predicción
    c2v_prediction_weighted_score: float # La lista de predicciones obtenidas por code2vec


iterator = zip(
            original_methods,
            masked_validate_functions,
            method_names,
            our_parsed_predictions,
            c2v_parsed_predictions)

data = []

for method_str, masked_method, method_name, our_prediction, c2v_prediction in iterator:
    parsed_method_name = parse_cammel_case(method_name)
    prediction_scores = score_prediction_result(our_prediction, parsed_method_name)
    c2v_prediction_scores = score_prediction_result(c2v_prediction, parsed_method_name)

    prediction_weighted_score = weighted_score_prediction_result(our_prediction, parsed_method_name)
    c2v_prediction_weighted_score = weighted_score_prediction_result(c2v_prediction, parsed_method_name)

    data.append(
        PredictionResult(
            method_str,
            masked_method,
            method_name,
            our_prediction,
            prediction_scores,
            prediction_weighted_score,
            c2v_prediction,
            c2v_prediction_scores,
            c2v_prediction_weighted_score
        )
    )

dataframe = DataFrame(data)
dataframe

,method_str,masked_method,method_name,prediction,prediction_scores,prediction_weighted_score,c2v_pred,c2v_prediction_scores,c2v_prediction_weighted_score
0,protected static long getCPtr(SWIGTYPE_p_p_btS...,protected static long [MASK](SWIGTYPE_p_p_btSo...,getCPtr,"[{'value': ['get'], 'probability': 0.262008130...","[1.0, 0.5726487, 0.46117607, 0.40115592, 0.613...",0.331079,"[{'value': ['get', 'C', 'Ptr'], 'probability':...","[1.0, 0.43109724, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3...","[1.0, 0.43109724, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3..."
1,public T get() {try {return future.get();} cat...,public T [MASK]() {try {return future.[MASK]()...,get,"[{'value': ['execute'], 'probability': 0.12403...","[0.37713256, 1.0, 0.46303764, 0.62328804, 0.52...",0.220333,"[{'value': ['poll'], 'probability': 0.74979263...","[0.29275376, 0.40682167, 0.926832, 0.10443802,...","[0.29275376, 0.40682167, 0.926832, 0.10443802,..."
2,protected void update(float percent) {value = ...,protected void [MASK](float percent) {value = ...,update,"[{'value': ['value'], 'probability': 0.4759539...","[0.21691036, 0.33396447, 0.4158154, 0.19306125...",0.155717,"[{'value': ['interpolate'], 'probability': 0.7...","[-0.039974824, 0.37657535, 0.37319824, 0.46057...","[-0.039974824, 0.37657535, 0.37319824, 0.46057..."
3,public int getValue() {return value;}\n,public int [MASK]() {return value;}\n,getValue,"[{'value': ['value'], 'probability': 0.9087800...","[0.46303764, 1.0, 0.43109724, 0.2945777, 0.453...",0.441420,"[{'value': ['get', 'value'], 'probability': 0....","[0.8500731, 0.46303764, 0.33695218, 0.87729096...","[0.8500731, 0.46303764, 0.33695218, 0.87729096..."
4,public void setValue(int value) {this.value = ...,public void [MASK](int value) {this.value = va...,setValue,"[{'value': ['set'], 'probability': 0.249739855...","[1.0, 0.7944568, 0.47274652, 0.31490386, 0.437...",0.506841,"[{'value': ['set', 'value'], 'probability': 0....","[0.8399167, 1.0, 0.7749108, 0.903226, 0.572824...","[0.8399167, 1.0, 0.7749108, 0.903226, 0.572824..."
...,...,...,...,...,...,...,...,...,...
1767,public Matrix4 getProjectionMatrix() {return p...,public Matrix4 [MASK]() {return projectionMatr...,getProjectionMatrix,"[{'value': ['##2'], 'probability': 0.178443595...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,"[{'value': ['get', 'projection', 'matrix'], 'p...","[0.5902986, 0.38085112, 0.70646435, 0.73254496...","[0.5902986, 0.38085112, 0.70646435, 0.73254496..."
1768,public void setProjectionMatrix(Matrix4 projec...,public void [MASK](Matrix4 projection) {if (dr...,setProjectionMatrix,"[{'value': ['set'], 'probability': 0.329746812...","[1.0, 0.6315808, 0.3280287, 0.48556754, 0.7944...",0.574687,"[{'value': ['set', 'projection', 'matrix'], 'p...","[0.67465353, 0.57497877, 0.80272853, 0.3492521...","[0.67465353, 0.57497877, 0.80272853, 0.3492521..."
1769,public Matrix4 getTransformMatrix() {return tr...,public Matrix4 [MASK]() {return transformMatri...,getTransformMatrix,"[{'value': ['transform'], 'probability': 0.137...","[0.33052462, 0.0, 0.0, 0.0, 0.0]",0.045412,"[{'value': ['get', 'transform', 'matrix'], 'pr...","[0.6747574, 0.8550423, 0.73254496, 0.5653039, ...","[0.6747574, 0.8550423, 0.73254496, 0.5653039, ..."
1770,public void setTransformMatrix(Matrix4 transfo...,public void [MASK](Matrix4 transform) {if (dra...,setTransformMatrix,"[{'value': ['set'], 'probability': 0.325132638...","[1.0, 0.3280287, 0.6315808, 0.40315592, 0.5010...",0.511842,"[{'value': ['get', 'transform', 'matrix'], 'pr...","[0.5933911, 0.40315592, 0.85847646, 0.41854778...","[0.5933911, 0.40315592, 0.85847646, 0.41854778..."


In [ ]:
dataframe.to_csv("./results.csv")